<a href="https://colab.research.google.com/github/MoonEeSun/Artificial_Intelligence/blob/main/HW4_Cliff_Walking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[인공지능] 과제4 Cliff Walking 예제 구현**
*   **QLearning Class 및 Sarsa Class를 완성하여 결과를 살펴보는 것이 목표**입니다.
*   기본적인 코드는 아래 노트에 모두 작성되어 있습니다. 비어있는 함수 부분을 완성하면 됩니다.
*   **과제 수행 시 주의사항: 외부 라이브러리로 Q-learning 및 Sarsa 적용하지 말 것, 수업 때 배운 내용대로 Q-learning과 Sarsa를 주어진 함수에 구현할 것.** 웹 상에 있는 다양한 Q-learning 및 Sarsa 코드를 참고하는 것은 괜찮습니다.
*   **보고서 작성 내용**: 여러분이 완성한 Q-learning 및 sarsa 알고리즘의 내용과 결과의 의미를 분석하는 내용을 작성하면 됩니다.
작성한 코드와 실행 결과를 첨부하길 바라며, 코드에는 자세한 주석을 필수적으로 포함하기 바랍니다. 보고서는 PDF로 제출바랍니다.
*   보고서는 11월 30일 오후 11시 59분까지 블랙보드에 보고서 형태로 제출하면 됩니다. 지각은 0점입니다.

# **본 노트를 본인의 drive로 복사하여 활용하기 바랍니다.**

본 과제는 OpenAI Gym 환경에 기반하여 작성되었습니다. Gym 라이브러리는 학습을 적용할 수 있는 다양한 환경을 제공합니다. 여기서는 수업에서 다뤘던 Cliff Walking 환경을 활용합니다.

In [ ]:
import gym
import matplotlib
import numpy as np
import random
import itertools # looping을 위해
import sys
from collections import defaultdict # dictionary의 key 값이 없을 경우 미리 지정해놓은 초기(default)값을 반환
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv # Cliff Walking 환경

QtoPolicy Class는 학습된 Q-value를 입력하면 해당하는 Q-value의 greedy 정책이 출력되도록 하는 함수 `printPolicy`를 구성하는 Class입니다. Q-learning 및 Sarsa를 이용하여 학습된 정책을 출력하기 위해 필요합니다.

In [ ]:
# 학습된 Q-value를 입력하면 그 Q-value의 greedy 정책이 출력되도록 함
# Q-learning이나 Sarsa를 통해 학습된 정책을 출력하기 위해 필요
class QtoPolicy:
    def __init__(self):
        self.action = ['↑','→','↓','←'] # Agent의 가능한 Action
    
    def printPolicy(self, Q):
        # Agent의 정책 : 학습된 Q-value가 Q-table에 있으면 Q-value를 가장 크게 만드는 정책을 저장(greedy한 정책)
        policy = np.array([np.argmax(Q[key]) if key in Q else -1 for key in np.arange(48)]) 
        # 가장 큰 Q-value : 학습된 Q-value가 Q-table에 있으면 Q-value 중 max값을, 없으면 0을 변수 v에 저장
        v = ([np.max(Q[key]) if key in Q else 0 for key in np.arange(48)])
        # Agent의 행동 저장
        actions = np.stack([self.action for _ in range(len(policy))], axis=0)
        
        # Agent의 행동에 따른 정책 출력
        print(np.take(actions, np.reshape(policy, (4, 12))))
        print('')

In [ ]:
# class QLearning:
#   # Q(s, a) 임의로 초기화
#     def __init__(self):
#         self.action_no = 4 # 행동 가짓수
#         self.alpha = 0.01 # learning rate
#         self.gamma = 0.9 # discount vector
#         self.epsilon = 0.2 # 앱실론
#         self.q_values = defaultdict(lambda: [0.0] * self.action_no) # value function : 누적보상
        
#     # update : Q-value Update (state(S), action(A), reward(R), next_state(S'), next_action(A') 주어짐)
#     def update(self, state, action, reward, next_state, next_action):
#         ## 함수를 완성하시오.
#         # Choose action        
#         # action = egreedy_policy(q_values, state, epsilon=0.1)
#         # Do the action
#         # next_state, reward, done = env.step(action)
#         # Update q_values     
#         # td_target = reward + self.gamma * np.max(q_values[next_state])   
#         td_target = reward + self.gamma * np.max(self.q_values[next_state][action])
#         td_error = td_target - self.q_values[state][action]
#         self.q_values[state][action] += self.alpha * td_error
#         # Update state
#         # state = next_state
    
#     # act : 앱실론 greedy에 따라 action 선택
#     def act(self, state):
#         ## 함수를 완성하시오.
#         # Get a random number from a uniform distribution between 0 and 1,
#         # if the number is lower than epsilon choose a random action
#         if np.random.random() < self.epsilon:
#           action = np.random.choice(4)
#         # Else choose the action with the highest value
#         else:
#           action = np.argmax(self.q_values[state]) # Choose the action with largest Q-value (state value)
#         return action

아래는 Q-learning 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.



In [ ]:
class QLearning:
  # Q(s, a) 임의로 초기화
    def __init__(self):
        self.action_no = 4 # 행동 가짓수
        self.alpha = 0.01 # learning rate
        self.gamma = 0.9 # discount vector
        self.epsilon = 0.2 # 앱실론
        self.q_values = defaultdict(lambda: [0.0] * self.action_no) # value function : 누적보상
        
    # update : Q-value Update (state(S), action(A), reward(R), next_state(S'), next_action(A') 주어짐)
    def update(self, state, action, reward, next_state, next_action): 
      # Q(S, A) <- Q(S, A) + alpha*[R + gamma*maxa(Q(S', a))-Q(S, A)]
        q_target = reward + self.gamma * np.max(self.q_values[next_state])
        q_error = q_target - self.q_values[state][action]
        self.q_values[state][action] += self.alpha * q_error
    
    # act : 앱실론 greedy에 따라 action 선택
    def act(self, state):
        # np.random.random() : 0과 1사이의 uniform distribution에서부터 random한 숫자를 가져오기
        # 숫자가 epsilon보다 작을 경우 랜덤하게 action을 선택
        if np.random.random() < self.epsilon:
          action = np.random.choice(4)
        # 숫자가 epsilon보다 크거나 같을 경우 q-value을 최대화하는 action을 선택
        else:
          action = np.argmax(self.q_values[state])
        return action

아래는 Sarsa 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.



In [ ]:
class Sarsa:
    def __init__(self):
        self.action_no = 4 # 행동의 가짓수
        self.alpha = 0.01 # learning rate
        self.gamma = 0.9 # discount vector
        self.epsilon = 0.2 # 앱실론
        self.q_values = defaultdict(lambda: [0.0] * self.action_no) # 가치함수
        
    # update : Q-value update (state, action, reward, next_tate, next_action 주어짐)
    def update(self, state, action, reward, next_state, next_action):
        # Q(S,A) <- Q(S,A) + alpha*[R+gamma*Q(S',A')-Q(S,A)]
        q_target = reward + self.gamma * self.q_values[next_state][next_action]
        q_error = q_target - self.q_values[state][action]
        self.q_values[state][action] += self.alpha * q_error
        
    # act : 앱실론 greedy에 따라 action 선택
    def act(self, state):
        # np.random.random() : 0과 1사이의 uniform distribution에서부터 random한 숫자를 가져오기
        # 숫자가 epsilon보다 작을 경우 랜덤하게 action을 선택
        if np.random.random() < self.epsilon:
          action = np.random.choice(4)
        # 숫자가 epsilon보다 크거나 같을 경우 q-value을 최대화하는 action을 선택
        else:
          action = np.argmax(self.q_values[state])
        return action

OpenAI Gym에서의 Cliff Walking 환경을 로드하고 해당하는 환경을 살펴보기 위해 `render()` 메쏘드를 사용해봅니다.
그리고 `env.nS` 및 `env.nA` 변수를 통해 해당 환경의 state 및 action 개수를 확인합니다.

Cliff Walking 환경에서 각 state는 grid에서의 위치, action은 'up', 'right', 'down', 'left' 방향을 의미합니다.

In [ ]:
env = CliffWalkingEnv() # Cliff Walking 환경 load
env.render() # render() : Graphic User Interface (GUI)로 현재 진행상황을 출력하는 함수
print ('Number of states: ', env.nS) # 해당 환경의 state
print ('Number of actions :', env.nA) # action 개수 - '↑','↓','→','←'

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Number of states:  48
Number of actions : 4


주어진 Q-value에서 greedy policy를 출력하는 QtoPolicy Class를 정의합니다.

In [ ]:
policy = QtoPolicy() # Q-value에서의 greedy 정책 출력을 위해 class 정의

Q-learning Class를 정의하고 5000 episode 동안 학습을 수행합니다.

Gym 라이브러리의 환경에서는 `step(action)` 메쏘드를 통해 해당하는 time-step에서 action을 수행한 효과를 얻을 수 있습니다. 해당 메쏘드에서는 action을 수행하여 얻어지는 보상 (reward), 다음 상태 (next_state), done (episode 종료여부) 등이 출력으로 주어집니다.

In [ ]:
agent_QL = QLearning() # Q-learning class 정의
for ep in range(5000):
    done = False
    # reset() : 상태 초기화 - Step을 실행하다가 epsiode가 끝나서 이를 초기화해서 재시작해야할 때, 초기 State를 반환
    state = env.reset() 
    action = agent_QL.act(state) # 앱실론 greedy 정책으로 Choose action
    
    ep_rewards = 0 # 총 누적 reward 변수 초기화
    while not done:
        # 알고리즘을 통해 얻어낸 Action을 1 step 수행한 후, 현재 state와 reward 등의 정보를 반환
        # Step()함수의 return : tuple 형태의 (next_state, reward, done, info)를 반환
        next_state, reward, done, info = env.step(action) 

        # 앱실론 greedy 정책으로 다음 action choose
        next_action = agent_QL.act(next_state) 

        # 가치함수 update
        agent_QL.update(state, action, reward, next_state, next_action) 
        
        ep_rewards += reward # 얻은 reward 더해줌
        state = next_state # 상태 update
        action = next_action # action update

In [ ]:
print('Q-learning reward :')
print(ep_rewards)

Q-learning reward :
-122


Sarsa에 대해서도 같은 방식으로 학습을 수행합니다.

In [ ]:
agent_Sa = Sarsa()
for ep in range(5000):
    done = False
    # reset() : 상태 초기화 - Step을 실행하다가 epsiode가 끝나서 이를 초기화해서 재시작해야할 때, 초기 State를 반환
    state = env.reset()
    action = agent_Sa.act(state) # 앱실론 greedy 정책으로 Choose action
    
    ep_rewards = 0 # 총 누적 reward 변수 초기화
    while not done:
        # 알고리즘을 통해 얻어낸 Action을 1 step 수행한 후, 현재 state와 reward 등의 정보를 반환
        # Step()함수의 return : tuple 형태의 (next_state, reward, done, info)를 반환
        next_state, reward, done, info = env.step(action) # action 수행

        # 앱실론 greedy 정책으로 다음 action choose
        next_action = agent_Sa.act(next_state) 

        # 가치함수 update
        agent_Sa.update(state, action, reward, next_state, next_action) 
        
        ep_rewards += reward # 얻은 reward 더해줌
        state = next_state # 상태 update
        action = next_action # action update

In [ ]:
print('Sarsa reward :')
print(ep_rewards)

Sarsa reward :
-19


학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [ ]:
print('Learned policy by Q-learning')
policy.printPolicy(agent_QL.q_values)
print('Learned policy by Sarsa')
policy.printPolicy(agent_Sa.q_values)

Learned policy by Q-learning
[['→' '↑' '→' '↓' '→' '↓' '→' '↓' '→' '→' '↓' '↓']
 ['←' '→' '→' '↓' '→' '→' '→' '→' '→' '→' '↓' '↓']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' '↑']]

Learned policy by Sarsa
[['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['→' '↑' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '←' '↑' '↑' '→' '↑' '↑' '↑' '↑' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' '↑']]



In [ ]:
env.close()